# Оптимизация в задачах машинного и глубокого обучения

Во время решения задач машинного обучения мы формулировали оптимизационную задачу: мы вводили некую функцию потерь и пытались с помощью некоторого алгоритма найти оптимальные параметры $w$, минимизирующие функционал качества. Возьмем, например, знакомый лосс:
$$
\left||y-Xw\right||^2_2 \rightarrow min
$$
Прелесть этой функции в том, что она является **выпуклой**:

Функция $f: \mathbb{R}^d \rightarrow \mathbb{R}$ является (нестрого) выпуклой (вниз), если для любых $x_1, x_2 \in \mathbb{R}^d$ верно, что
$$
\forall\theta \in [0,1]: f(\theta x_1+(1-\theta)x_2)\leq \theta f(x_1) + (1-\theta) f(x_2)
$$

<center><img src="data//convex_function.png" alt="drawing" width="700"/></center>


Важное свойство выпуклых функций – локальный минимум автоматически является глобальным. Однако помимо "хороших", выпуклых функций часто встречаются и невыпуклые. Например, функция потерь при обучении нейронных сетей, как правило, не является выпуклой. Найти глобальный минимум невыпуклой функции – очень трудная задача, но зачастую нам хватает локального, который является, в частности, стационарной точкой: такой, в которой производная равна нулю. Все теоретические результаты в случае невыпуклых задач, как правило, касаются поиска таких точек, и алгоритмы тоже направлены на их отыскание. Этим объясняется и то, что большинство алгоритмов оптимизации, придуманных для выпуклого случая, дословно перешли в невыпуклый. Теоретическая причина в следующем: в выпуклом случае поиск стационарной точки и поиск минимума – буквально *одна и та же задача*, поэтому то, что хорошо ищет минимум в выпуклом случае, ожидаемо будет хорошо искать стационарные точки в невыпуклом. 

Несмотря на то, что для произвольных функций мы зачастую будем искать локальный минимум (найти глобальный минимум может не удастся), мы все равно будем использовать те же алгоритмы выпуклой оптимизации, поскольку:
- сойтись в локальный минимум лучше, чем никуда, тем более локальный минимум может нас вполне устроить (значение функции потерь может быть достаточно малым в этой точке, несмотря на то, что существует еще и глобальный минимум)
- в окрестности локального минимума функция становится выпуклой, и там мы сможем быстро сойтись

Стоит отметить важную разницу между выпуклой и невыпуклой задачами: в выпуклом случае работа алгоритма оптимизации не очень существенно зависит от начальной точки, поскольку мы всегда скатимся в точку оптимума. В невыпуклом же случае правильно выбранная точка старта – это уже половина успеха.

### Алгоритмы оптимизации

- **Градиентный спуск (Gradient Descent)**

Необходимо вспомнить, что **градиент** - это вектор, направленный в сторону наискорейшего локального возрастания функции. В таком случае **антиградиент** (градиент взятый со знаком минус) - направление наискорейшего локального убывания. На каждой итерации будем двигаться в сторону антиградиента и обновлять значение весов:
\begin{align*}
    w^{(k+1)}=w^{(k)}-\lambda\nabla_wL(w^{(k)}),
\end{align*}
где $\lambda$ - гиперпараметр, который называется **темпом обучения** (learning rate). С его помощью можно регулировать величину шага в направлении антиградиента.
<center><img src="data\\grad_desc.png" alt="drawing" width="500"/></center>

Хранение градиентов в памяти может доставить нам проблемы. У градиента столько же компонент, сколько параметров у модели, и если мы имеем дело с глубокой нейросетью, это даст значительные затраты дополнительной памяти.

Также рассмотрим, за сколько шагов можно достичь желаемого качества. В максимально общем выпуклом случае без дополнительных предположений оценки для градиентного спуска крайне пессемистичные: чтобы достичь качества $\varepsilon$, то есть 
$$
\left|f(x_k)-f(x^\ast) \right| \leq \varepsilon
$$
достаточно сделать $O\left(R^2/\varepsilon^2\right)$  шагов, где $R^2$ -- это расстояние от $x_0$ до $x_1$. Чтобы достичь точности $10^{-2}$, необходимо сделать порядка $10^4$ шагов градиентного спуска, что довольно много. Но на практике такого не происходит, потому что на самом деле верны разные предположения, дающие более приятные свойства. Для контраста, укажем оценку в случае гладкой и сильно выпуклой в точке оптимума функции: за $k$ шагов будет достигнута точность
$$
O\left(\min\left[R^2exp\left(-\frac{k}{4\kappa}\right), \frac{R^2}{k} \right]  \right)
$$
где $\kappa$ - число обусловленности задачи. По сути, это число измеряет, насколько линии уровня функции вытянуты в окрестности оптимума. Таким образом, скорость сходимости градиентного спуска сильно зависит от обусловленности задачи, а также от выбора начальной точки, ведь во всех оценках присутствует расстояние от точки старта до оптимума.


- **Стохастический градиентный спуск (Stochastic gradient descent, SGD)**

Рассмотрим функционал вида
$$
f(x) = \frac{1}{N}\sum\limits_{i=1}^N L(x, y_i)
$$
заметим, что это усреднение, то есть по сути взятие матожидания. Таким образом, мы говорим, что наша функция выглядит так:
$$
f(x) = \mathbb{E}\left[ L(x, \xi) \right]
$$
где $\xi$ равномерно распределена по обучающей выборке. Для функционалов такого вида мы также можем посчитать градиент:
$$
\nabla f(x) = \mathbb{E} \nabla L(x, \xi)
$$

После этого матожидание заменяется на его несмещенную Монте-Карло оценку. Получается то, что можно назвать **стохастическим градиентом**:
$$
\tilde {\nabla} f(x) = \frac{1}{B}\sum\limits_{i=1}^B  \nabla L(x, \xi_i)
$$
мы таким образом заменили вычисление градиента по всей выборке вычислением по случайной *подвыборке*. Подвыборку $\xi_1, \dots, \xi_B$ часто называют **(мини)батчем**, а число $B$ – размерном батча.

По-хорошему нужно каждый раз независимо генерировать батчи, но это трудно с вычислительной точки зрения. Вместо этого поступают следующим приёмом: сначала перемешивают выборку (чтобы внести дополнительную случайность), а затем рассматривают последовательно блоки по $B$ элементов выборки. Когда мы просмотрели всю выборку – перемешиваем еще раз и повторяем проход. Очередной прогон по обучающей выборке в глубоком обучении называется *эпохой*. 

Алгоритм выглядит следующим образом
```python
x = normal(0, 1)                    # инициализация
repeat E times:                     # цикл по количеству эпох
   for i = 0; i <= N; i += B:
        batch = data[i:i+B]
        h = grad_loss(batch).mean() # вычисляем оценку градиента как среднее по батчу
        x -= alpha * h
```

Дополнительное удобство такого подхода – возможность работы с внешней памятью, ведь выборка может быть настолько большой, что она помещается только на жёсткий диск. Поскольку стохастические градиенты являются лишь оценками истинных градиентов, SGD может быть довольно шумным:

<center><img src="data\\sgd.png" alt="drawing" width="700"/></center>

- **Метод инерции, momentum**
  
Мы до сих пор не использовали информацию о предыдущих шагах алгоритма. Идея в том, что используя такую информацию можно улучшить сходимость алгоритма.

Метод инерции можно объяснить с помощью физической аналогии. Функцию потерь можно представить как холм или гору, а на сам процесс оптимизации можно смотреть как на моделирование значений вектора параметров частицы (мячика), катящейся с холма. Реальный мячик не застрянет перед небольшой кочкой, так как у него есть некоторая масса и уже накопленный импульс – некоторое время он способен двигаться даже вверх по склону. Аналогичный прием может быть использован и в градиентной оптимизации.

С математической точки зрения, мы добавляем к градиентному шагу еще одно слагаемое:
$$
x_{k+1}=x_k-\alpha_k\nabla f(x_k) + \beta_k (x_k-x_{k-1})
$$
Видно, что появился еще один гиперпараметр $\beta_k$. Выгода в том, что мы будем пропускать паразитные локальные минимумы и седла и продолжать движение вниз. Также удобно представить метод в виде двух параллельных итерационных процессов, используя для удобства обозначение для скорости (импульса) $\vartheta$:
$$
\vartheta_{k+1}=\beta_k\vartheta_k-\alpha_k\nabla f(x_k)
$$
$$
x_{k+1} = x_k + \vartheta_{k+1}
$$
- **Accelerated Gradient Descent (Nesterov Momentum)**

Можно доказать, что в сильно выпуклом и гладком случае найти минимум с точностью $\varepsilon$ нельзя быстрее, чем за
$R^2exp\left(-\frac{k}{\sqrt{\kappa}} \right)$ итераций. Для обычного градиентного спуска в экспоненте у нас был не корень из $\kappa$, а просто $\kappa$, то есть, градиентный спуск справляется с плохой обусловленностью задачи хуже, чем мог бы.

В 1983 году Ю. Нестеровым был предложен алгоритм, имеющий оптимальную по порядку оценку. Для этого модифицируем немного импульс и будем считать градиент не в текущей точке, а как бы в точке, в которую мы бы пошли, следуя импульсу:
$$
\vartheta_{k+1}=\beta_k\vartheta_k-\alpha_k\nabla f(x_k+\beta_k\vartheta_k)
$$
$$
x_{k+1} = x_k + \vartheta_{k+1}
$$

<center><img src="data\\momentum.png" alt="drawing" width="800"/></center>

Nesterov Momentum позволяет значительно повысить устойчивость и скорость сходимости в некоторых случаях. Но, конечно, он не является серебряной пулей в задачах оптимизации, хотя в выпуклом мире и является теоретически неулучшаемым.

**Адаптивный подбор размера шага**

Можно пойти дальше и задаться вопросом подбора размера шага. 

- **Adagrad**

Adagrad является адаптацией стохастического градиентного спуска, впервые алгоритм был предложен в 2011 году. Зафиксируем исходный learning rate $\alpha$. Затем напишем следующую формулу обновления:
$$
G_{k+1}=G_k+\left(\nabla f(x_k) \right)^2
$$
$$
x_{k+1} = x_k-\frac{\alpha}{\sqrt{G_{k+1}+\varepsilon}}\nabla f(x_k)
$$
Возведение в квадрат и деления векторов покомпонентные. Наш размера шага для фиксированной координаты – это какая-то изначальная константа $\alpha$ (learning rate), деленная на корень из суммы квадратов координат градиентов плюс дополнительный параметр сглаживания $\varepsilon$, предотвращающий деление на ноль. Эта добавка на практике обычно фиксируется равной дефолтными $10^{-8}$ и не изменяется. 

**Идея алгоритма следующая**: если мы вышли на плато по какой-то координате и соответствующая компонента градиента начала затухать, то нам нельзя уменьшать размер шага слишком сильно, поскольку мы рискуем на этом плато остаться, но в то же время уменьшать надо, потому что это плато может содержать оптимум. Если же градиент долгое время довольно большой, то это может быть знаком, что нам нужно уменьшить размер шага, чтобы не пропустить оптимум. Поэтому мы стараемся компенсировать слишком большие или слишком маленькие координаты градиента. Однако иногда получается так, что величина шага слишком быстро уменьшается; для решения этой проблемы придумали другой алгоритм.

- **RMSProp**

Теперь будем не просто складывать нормы градиентов, а усреднять их в скользящем режиме:
$$
G_{k+1}=\gamma G_k+ (1-\gamma) \left(\nabla f(x_k) \right)^2
$$
$$
x_{k+1} = x_k-\frac{\alpha}{\sqrt{G_{k+1}+\varepsilon}}\nabla f(x_k)
$$
Такой выбор позволяет все еще учитывать историю градиентов, но при этом размер шага уменьшается не так быстро. Благодаря адаптивному подбору шага в современных оптимизаторах не нужно подбирать последовательность $\alpha_k$ размеров всех шагов, а достаточно выбрать всего одно число – learning rate $\alpha$, всё остальное сделает за вас сам алгоритм. Но learning rate все еще нужно выбирать крайне аккуратно: алгоритм может либо преждевременно выйти на плато, либо вовсе разойтись.

<center><img src="data\\learningrates.jpeg" alt="drawing" width="400"/></center>


- **Adam (ADAptive Momentum)**
  
Для описания работы Adam объединим идеи предыдущих алгоритмов. Выражения для обновления параметров выглядят так:

$$
\vartheta_{k+1}=\beta_1\vartheta_k+(1-\beta_1)\nabla f(x_k)
$$
$$
G_{k+1}=\beta_2 G_k+(1-\beta_2)\left(\nabla f(x_k) \right)^2
$$
$$
x_{k+1} = x_k-\frac{\alpha}{\sqrt{G_{k+1}+\varepsilon}}\vartheta_{k+1}
$$
Как правило, в этом алгоритме подбирают лишь один гиперпараметр $\alpha$ – learning rate. Остальные же: $\beta_1$, $\beta_2$ и $\varepsilon$ – оставляют стандартными и равными $0.9$, $0.99$ и $10^{-8}$ соответственно. Основная сложность, ествествено, заключается в подборе наилучшего темпа обучения.

Adam является сильным оптимизационным алгоритмом и зачастую используется как оптимизатор по умолчанию. Надо только отметить, что Adam требует хранения как параметров модели, так и градиентов, накопленного импульса и нормировочных констант. Т.е. достижение более быстрой (с точки зрения количества итераций/объема рассмотренных данных) сходимости требует больших объемов памяти.

Визуализация работы различных оптимизаторов             |  Визуализация поведения алгоритмов в седловой точке
:-------------------------:|:-------------------------:
<img src="data\\opt2.gif" alt="drawing" width="500"/> | <img src="data\\opt1.gif" alt="drawing" width="500"/>

Источник анимаций: CS231n Convolutional Neural Networks for Visual Recognition